In [1]:
# `cnn_4_llm_memo`

# Copyright 2025-present Laboratoire d'Informatique de Polytechnique.
# Apache Licence v2.0.

import os

os.chdir(os.path.dirname(os.path.abspath('')))

# Extract patterns

This notebook helps you localize the patterns and detect IDs you need, and extract a small proportion of attention patterns to analyze them locally.

## Localizing patterns

First, you need to get the IDs of the experiments to analyze.

In [2]:
import src

In [3]:
from src.patterns import PatternsConfig
from src.detect import DetectConfig

from pathlib import Path

In [4]:
#TARGET_TAX = "merge_2_other_code_recollect_recite"
TARGET_TAX = "merge_2_other_guess_recollect_recite"

In [7]:
saved_id = None
for child in Path("/Users/jeremie/Documents/01-Travail/01-Doctorat/regu-detect/output/patterns").iterdir():
    cfg = PatternsConfig.from_json(child / "config.json", reset_output_dir=True)
    if cfg.tax_name == TARGET_TAX and cfg.size == "12b" and cfg.duplicates_threshold == 50:
        saved_id = cfg.get_id()
        print(saved_id)

rasyq52cr19yVj72U1ZkFQ


In [8]:
saved_detect_id = None
for child in Path("/Users/jeremie/Documents/01-Travail/01-Doctorat/regu-detect/output/detect").iterdir():
    cfg = DetectConfig.from_json(child / "config.json", reset_output_dir=True)
    if cfg.patterns_config == saved_id:# and cfg.kernel_size == 6 and cfg.n_feat_cnn == 10 and cfg.head_pooling == "max":
        saved_detect_id = cfg.get_id()
        print(saved_detect_id)

T31DI1_xGRSqwVnaYg8RQA
njf0kB4Oc0SuUStqC_T_gg
j-55nwPUR6a3jK9uFnkeKg
UjdbeBW91ldG1Qz2_USWTg
4S9AQiT_hRv-u0DhEWUIBg
3nhzGHe8LQTthAVVg0kDqQ
_oz-DWAzlH4j8_LBpUzUsw
A-nLMe3_FC9MGKF93jlRWA


## Extracting patterns

Now, you can use these detect IDs and patterns IDs to extact the correct patterns. For example, notebook `05-interpretation_rasy` gives you `A-nL.pickle` which contains a list of sequence IDs to extract. Copy this file on the machine that stores the attention patterns, and run the following lines.

In [5]:
import h5py
import numpy as np
import pickle

In [6]:
# A-nL
pickle_path = "/lustre/fsn1/projects/rech/yfw/upp42qa/h5management/A-nL.pickle"
h5_input_path = "/lustre/fsn1/projects/rech/yfw/upp42qa/h5management/attention_patterns_rasy.h5py"
h5_output_path = "/lustre/fsn1/projects/rech/yfw/upp42qa/h5management/attention_patterns_rasy_extract.h5py"

In [10]:
with open(pickle_path, "rb") as f:
    sequence_list = pickle.load(f)

print(len(sequence_list))
type(sequence_list[0])

300


numpy.int64

In [ ]:
with h5py.File(h5_output_path, "w") as h5f_output:
    with h5py.File(h5_input_path, "r") as h5f_input:
        for sequence_id in sequence_list:
            attention_pattern = h5f_input[str(sequence_id)][()]  # Load as np array
            h5f_output.create_dataset(
                str(sequence_id),
                data=attention_pattern.astype(np.float16),
            )